In [ ]:
indeed65kds = pd.read_csv('sample_df_indeed65k_datascientist.csv')

In [3]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

# Scraping the data from Indeed.com

In [6]:
URL = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
#print(soup.prettify())

In [7]:
def extract_job_title_from_result(soup):
    
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
          jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

[u'R&D Scientist, Jr.',
 u'Data Scientist II',
 u'Internship--Data Analysis & Research (Ref # LAB_DA/RES_VUGS_FA/SP_2018-19)',
 u'Analyst - Healthcare Analytics & Delivery, New Ventures (Contract)',
 u'Data Scientist',
 u'Assistant or Associate Research Support Specialist, Document Analysis',
 u'Marketing Analytics Associate',
 u'AI Engineer - Machine Learning - New York, US',
 u'Data Scientist (New York)',
 u'Jr. Data Scientist',
 u'Analytics Expert - Journey Analytics, New Ventures',
 u'Data Analyst',
 u'Associate Research Scientist',
 u'Quant Developer / Data Scientist',
 u'Laboratory Scientist Position\u2013 Drug Testing/Forensic Toxicology']

In [8]:
def extract_company_from_result(soup):
    
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
extract_company_from_result(soup)

[u'Chembio Diagnostic Systems, Inc.',
 u'Paychex',
 u'New York State Office of the Attorney General (OAG...',
 u'McKinsey & Company',
 u'NIAH Recruiting',
 u'Cary Institute of Ecosystem Studies',
 u'Via',
 u'BenevolentAI',
 u'Scienaptic, Inc.',
 u'BigR.io',
 u'McKinsey & Company',
 u'Schireson',
 u'NYU Langone Health',
 u'GMS Advisors',
 u'Champlain Toxicology Lab']

In [9]:
def extract_location_from_result(soup):
    
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

[u'Medford, NY 11763',
 u'Rochester, NY',
 u'New York, NY',
 u'New York, NY 10022 (Midtown area)',
 u'New York, NY',
 u'Millbrook, NY',
 u'New York, NY',
 u'New York, NY',
 u'New York, NY',
 u'New York, NY',
 u'New York, NY 10022 (Midtown area)',
 u'New York, NY',
 u'New York, NY 10016',
 u'New York, NY 10018',
 u'Plattsburgh, NY 12901']

In [11]:
def extract_salary_from_result(soup): 
    
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find("nobr").text)
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"sjcl"})
                div_three = div_two.find("div")
                salaries.append(div_three.text.strip())
            except:
                salaries.append("Nothing_found")
    return(salaries)
extract_salary_from_result(soup)

['Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found']

In [12]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll("span", attrs={"class": "summary"})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
#extract_summary_from_result(soup)

In [14]:
max_results_per_city = 10 ##
city_set = ["New+York","Chicago","San+Francisco", "Austin", "Seattle", "Los+Angeles", "Philadelphia", "Atlanta", "Dallas", "Pittsburgh", "Portland", "Phoenix", "Denver", "Houston", "Miami", "Washington+DC", "Boulder"]
columns = ["city", "job_title", "company_name", "location", "summary", "salary"]
sample_df = pd.DataFrame(columns = columns)

## In the following piece of code, the base salary is encoded between %24 and %2C symbols of the url. For example %2420%2C000 = pull job posts that have a 20000+ base salary. I changed it to %2465%2C000 to pull the data scientist positions that had $65,000+ salaries.  

In [15]:
for city in city_set:
    for start in range(0,max_results_per_city,10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start='+str(start))
        time.sleep(1)
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
        for div in soup.find_all(name="div",attrs={"class":"row"}):
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df)+1)
            #Creating an empty list to hold the data for each posting
            job_post = []
            #append city name
            job_post.append(city)
            # grabbing job title
            for a in div.find_all(name="a",attrs={"data-tn-element":"jobTitle"}):
                job_post.append(a["title"])
            #grabbing company name
            company = div.find_all(name="span", attrs={"class":"company"})
            if len(company) > 0:
                for b in company:
                    job_post.append(b.text.strip())
            else:
                sec_try = div.find_all(name="span",attrs={"class":"result-link-source"})
                for span in sec_try:
                    job_post.append(span.text)
            #grabbing location name
            c = div.findAll("span", attrs={"class":"location"})
            for span in c:
                job_post.append(span.text)
            #grabbing summary text
            d = div.findAll("span",attrs={"class":"summary"})
            for span in d:
                job_post.append(span.text.strip())
            #grabbing salary
            try:
                job_post.append(div.find('nobr').text)
            except:
                try:
                    div_two = div.find(name="div", attrs={"class":"sjcl"})
                    div_three = div_two.find("div")
                    job_post.append(div_three.text.strip())
                except:
                    job_post.append("Nothing Found")
            #appending list of job post info to dataframe at index num
            sample_df.loc[num] = job_post

    sample_df.to_csv("sample_df_indeedtest.csv", encoding="utf-8")

In [16]:
## The output DataFrame after scrapping data from Indeed.com
# City --> The job located at which city
# Job_title --> job title as it appears in the job post.
# Company_name --> Name of the employer.
# Location--> county or location of the job.
# summary --> Job Description.
# Salary --> If a salary is provided by the employer, it would have been grabbed and saved in the salary column
#            Most job posts do not provide a salary. Therefore a salary estimate is needed.
#
sample_df

,city,job_title,company_name,location,summary,salary
1,New+York,Laboratory Scientist Position– Drug Testing/Fo...,Champlain Toxicology Lab,"Plattsburgh, NY 12901",Laboratory Scientist Position– Drug Testing/Fo...,Nothing Found
2,New+York,Data Scientist II,Paychex,"Rochester, NY","5 years of experience in Data mining, data war...",Nothing Found
3,New+York,Associate Research Scientist,NYU Langone Health,"New York, NY 10016",May develop protocol for the collection and qu...,Nothing Found
4,New+York,Senior Data Scientist (Open to Remote),EXPERIAN,"New York, NY 10020 (Midtown area)",Experience analyzing large data set in order t...,Nothing Found
5,New+York,Data Scientist- Staff Associate,Columbia University Medical Center,"New York, NY 10032 (Washington Heights area)",We are looking for a new person to join our re...,Nothing Found
6,New+York,Data Scientist for Data Analytics Company,Huxley IT,"Manhattan, NY",We are seeking to add a Data Scientist to join...,Nothing Found
7,New+York,"Associate, Analytics",Xaxis,"New York, NY","Xaxis combines proprietary technology, unique ...",Nothing Found
8,New+York,Research Data Analyst,The Clark Hulings Fund for Visual Artists,New York State,Research Data Analyst. The Educational researc...,Nothing Found
9,New+York,Data Scientist - Internship,Dataminr,"New York, NY","Excellent knowledge of data structures, algori...",Nothing Found
10,New+York,"Analyst - Healthcare Analytics & Delivery, McK...",McKinsey & Company,"New York, NY 10022 (Midtown area)","It is a unique mix of Healthcare Experts, phys...",Nothing Found
